# Training loop for UNet with SE and res blocks
First training step is on patches of 512x512 cxrs, patch sizes should be 64x64.

## Flexible setup
This NB is designed to work flawlessly with either Google Colab or on a local machine.

In [12]:
# Colab setup
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print("IN_COLAB=", IN_COLAB)

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive/")
    os.chdir('/content/drive/MyDrive/Denoise/cxr-denoise/training')
    !pip install -r ../requirements.txt -q
    print("Requirements installed!")

IN_COLAB= False


In [13]:
# Enviorment setup
import os
import torch
import numpy as np
import random

# Seeding
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# TODO (optional): fix xpu support
print("Using:", device)


Using: cpu


In [14]:
# Get dataset from drive in colab - else sets DATA_PATH
# expect <5 min for 512², or <20 for 1024²
if IN_COLAB:
    import zipfile

    ZIP_PATH = "/content/drive/MyDrive/Datasets/NIH_data_512.zip"
    UNZIP_PATH = "/content/data/NIH_data_512"

    if not os.path.exists(UNZIP_PATH):
        os.makedirs(UNZIP_PATH, exist_ok=True)
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(UNZIP_PATH)
    DATA_PATH = "/content/data/NIH_data_512/content/NIH_data_512/" # TODO: fix redundent "content" - likely mistake in zipping
else:
    DATA_PATH = "../data/NIH_data_512"

## Dataset and Noise model setup
Sets up the splits for the diffrent NIH datasets

In [15]:
from nih_dataset import NIH_Dataset, RandomPatchDataset
from torch.utils.data import DataLoader
from noise import PreprocessClean, ApplyNoise
import torch
from PIL import Image
from torchvision import transforms

PRELOAD_TO_RAM = IN_COLAB

nih_train = NIH_Dataset(DATA_PATH, split="train", preload_to_ram=PRELOAD_TO_RAM)
nih_val = NIH_Dataset(DATA_PATH, split="val", preload_to_ram=PRELOAD_TO_RAM)
nih_test = NIH_Dataset(DATA_PATH, split="test", preload_to_ram=PRELOAD_TO_RAM)

LIMIT_DATA = None # For Using Full Dataset
#LIMIT_DATA = [320, 100, 100] # For Testing

if LIMIT_DATA:
    nih_train = torch.utils.data.Subset(nih_train, range(LIMIT_DATA[0])) if LIMIT_DATA[0] else nih_train
    nih_val = torch.utils.data.Subset(nih_val, range(LIMIT_DATA[1])) if LIMIT_DATA[1] else nih_val
    nih_test = torch.utils.data.Subset(nih_test, range(LIMIT_DATA[2])) if LIMIT_DATA[2] else nih_test

BATCH_SIZE = 4

train_dl = DataLoader(nih_train, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)
val_dl = DataLoader(nih_val, batch_size=BATCH_SIZE)
test_dl = DataLoader(nih_test, batch_size=BATCH_SIZE)

PATCH_BATCH_SIZE = 64
PATCH_SIZE = 64

patch_train_dl = DataLoader(RandomPatchDataset(nih_train, PATCH_SIZE), batch_size=PATCH_BATCH_SIZE, pin_memory=True, num_workers=4)

print(f"train/val/test: {len(nih_train)}/{len(nih_val)}/{len(nih_test)}")


NIH CXR Dataset (512x512), split='train' loaded with 69791 samples
NIH CXR Dataset (512x512), split='val' loaded with 16733 samples


KeyboardInterrupt: 

In [ ]:
# Noise model
preproc: PreprocessClean = PreprocessClean() # apply to clean images for ground truth
noiser: ApplyNoise = ApplyNoise(blur_sigma=1.0) # apply to preproc to get noisy features

sample = next(iter(train_dl))
print("Sample batch shape:", sample[0].shape)

# Load and transform images named example_01.png to example_06.png
example_transf = transforms.ToTensor()
example_patches = [
    example_transf(Image.open(f"examples/patch_{i:02d}.png").convert("L")) for i in range(1, 7)
]

# Example Patches
example_patch_batch = torch.stack(example_patches).to(device)  # shape: (6, 1, H, W)
import cxr_plt

example_clean = preproc(example_patch_batch)
example_noisy = noiser(example_clean)

cxr_plt.plot_cxr_images(
    [example_clean[0], example_noisy[0], example_clean[1], example_noisy[1]], 
    ["Clean 1", "Noisy 1", "Clean 2", "Noisy 2"], figsize=(4*4, 4))


## Model and training

In [ ]:
# Model
from unet_model import ResUNetSE
import torchsummary
import torchinfo
import torch

model = ResUNetSE(features=32, se_reduction=16).to(device=device)

with torch.no_grad():
    out = model(torch.zeros((1, 1, 512, 512)).to(device))
    print("model output shape:", out.shape)

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

/home/morris/Workspace/cxr-denoise/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model output shape: torch.Size([1, 1, 512, 512])
Trainable parameters: 8,169,711


In [ ]:
# Training loop - patch training

import torch
from cxr_plt import *
from tqdm import tqdm
from loss import CombinedLoss, SpatialLoss, FrequencyLoss
import matplotlib.pyplot as plt
import os
from torchvision.transforms.functional import to_pil_image
from torchvision.utils import make_grid

from torchmetrics.functional import structural_similarity_index_measure as ssim
import torch.nn.functional as F
from train_util import *
import csv

criterion = CombinedLoss(1.0, 0.3)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
model.train()

num_epochs = 10
patch_size = 64


metrics_log_path = "checkpoints/training_metrics.csv"
os.makedirs("checkpoints", exist_ok=True)

# Create CSV file with header if not exists
if not os.path.exists(metrics_log_path):
    with open(metrics_log_path, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "val_psnr", "val_ssim", "val_l1", "train_psnr", "train_ssim", "train_l1"])


total_batches = 0
for epoch in range(num_epochs):
    total_loss = 0
    pbar = tqdm(patch_train_dl, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in pbar:
        batch = batch[0] if isinstance(batch, (tuple, list)) else batch
        clean = batch.to(device)  # (B, 1, H, W)

        # Preprocess and noise
        clean = preproc(clean)

        # Apply noise
        noisy = noiser(clean)

        # Forward pass
        output = model(noisy)
        loss = criterion(output, clean)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_batches += 1

        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

        #if total_batches % 100 == 0:
        #    print("checkup!")
        #    plot_cxr_images([clean[0], noisy[0], output[0]], ["Clean", "Noisy", "Out"])

    # Log visuals and metrics after each epoch
    with torch.no_grad():
        model.eval()

        clean = preproc(example_patch_batch.to(device))
        noisy = noiser(clean)
        output = model(noisy)

        save_cxr_triplet(clean, noisy, output, f"example_patches_epoch_{epoch}.png")

        val_metrics = evaluate_metrics_on_dataloader(model, val_dl, preproc, noiser, device, max_batches=500)
        train_metrics = evaluate_metrics_on_dataloader(model, train_dl, preproc, noiser, device, max_batches=500)

        # Save model checkpoint
        torch.save(model.state_dict(), f"checkpoints/model_epoch_{epoch+1}.pt")

        # Write metrics to CSV
        with open(metrics_log_path, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1,
                val_metrics["psnr"],
                val_metrics["ssim"],
                val_metrics["l1"],
                train_metrics["psnr"],
                train_metrics["ssim"],
                train_metrics["l1"]
            ])


        model.train()

    avg_loss = total_loss / len(patch_train_dl)
    print(f"Epoch {epoch+1} Average Loss: {avg_loss:.4f}")

Epoch 1/10:   3%|▎         | 31/1091 [01:39<56:41,  3.21s/it, loss=0.1203]  


KeyboardInterrupt: 

## Save and Eval

In [ ]:
import time
import torch

model.train()
num_batches = 400
dataloader = patch_train_dl 

times = {
    "load": 0.0,
    "preprocess": 0.0,
    "forward": 0.0,
    "backward": 0.0,
    "optimizer": 0.0
}

data_iter = iter(dataloader)

for _ in range(num_batches):
    # --- Load from dataloader ---
    t0 = time.time()
    batch = next(data_iter)
    batch = batch[0] if isinstance(batch, (tuple, list)) else batch
    clean = batch.to(device)
    t1 = time.time()

    # --- Preprocess & Noise ---
    clean_pre = preproc(clean)
    noisy = noiser(clean_pre)
    t2 = time.time()

    # --- Forward ---
    output = model(noisy)
    loss = criterion(output, clean_pre)
    t3 = time.time()

    # --- Backward ---
    optimizer.zero_grad()
    loss.backward()
    t4 = time.time()

    # --- Optimizer Step ---
    optimizer.step()
    t5 = time.time()

    # Accumulate times
    times["load"] += t1 - t0
    times["preprocess"] += t2 - t1
    times["forward"] += t3 - t2
    times["backward"] += t4 - t3
    times["optimizer"] += t5 - t4

# Average
for key in times:
    times[key] /= num_batches

print("Profiling results (avg per batch):")
print(f" Load from dataloader: {times['load']:.4f} s")
print(f" Preprocess + noise:   {times['preprocess']:.4f} s")
print(f" Forward pass:         {times['forward']:.4f} s")
print(f" Backward pass:        {times['backward']:.4f} s")
print(f" Optimizer step:       {times['optimizer']:.4f} s")
print(f"-Total per step:       {sum(times.values()):.4f} s")

In [ ]:
# Save the model weights
MODEL_SAVE_PATH = "checkpoints/denoise_model_epoch_last.pth"
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)



torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch + 1,  # last completed epoch
    'patch_size': patch_size,
    'parameters': sum(p.numel() for p in model.parameters() if p.requires_grad),
}, MODEL_SAVE_PATH)

print(f"Model saved to {MODEL_SAVE_PATH}")

In [ ]:
# Test model